In [4]:
#建立DC要的io文件 CHIP.v
L = 10
for n in range(4, 10, 1):
    for r in range(3, 6, 1):
        m = n - r + 1
        if (m<2):
            break
        path = '../win_verilog/wc_%d_%d/' % (m, r)
        file = 'CHIP.v'
        outf = open(path + file, 'w')

        outf.write('//CHIP io for winograd\n')
        outf.write('module CHIP (clk, rst, D, Z);\n\n')
        outf.write('\tinput clk, rst;\n')
        outf.write('\tinput [%d*%d-1:0] D;\n' % (L,n))
        outf.write('\toutput [%d*%d-1:0] Z;\n\n' % (L,m))
        outf.write('\twire i_clk, i_rst;\n')
        outf.write('\twire [%d*%d-1:0] i_D;\n' % (L,n))
        outf.write('\twire [%d*%d-1:0] i_Z;\n\n' % (L,m))
        outf.write('\twire n_Logic0_, n_Logic1_;\n\n')
        outf.write('\tWC top (.rst(i_rst), .clk(i_clk), .D(i_D), .Z(i_Z));\n\n')

        outf.write('\tXMC ipad_CLK    ( .I(clk), .PU(n_Logic0_), .PD(n_Logic0_), .SMT(n_Logic0_), .O(i_clk) );\n')
        outf.write('\tXMC ipad_RESET_ ( .I(rst), .PU(n_Logic0_), .PD(n_Logic0_), .SMT(n_Logic0_), .O(i_rst) );\n\n')

        for i in range(L*n):
            outf.write('\tXMC ipad_D%d ( .I(D[%d]), .PU(n_Logic0_), .PD(n_Logic0_), .SMT(n_Logic0_), .O(i_D[%d]) );\n' % (i, i, i))
        outf.write('\n')

        for i in range(L*m):
            outf.write('\tYA2GSC opad_Z%d ( .I(i_Z[%d]), .E(n_Logic1_), .E2(n_Logic1_), .E4(n_Logic1_), .E8(n_Logic1_), .SR(n_Logic1_), .O(Z[%d]));\n' % (i, i, i))
        outf.write('\n')

        outf.write('\tTIE0 U1 ( .O(n_Logic0_) );\n')
        outf.write('\tTIE1 U2 ( .O(n_Logic1_) );\n\n')
        outf.write('endmodule\n')
        outf.close()

        #創建io_tdf.txt用於等等要做io.tdf
        file = 'io_tdf.txt'
        outf = open(path + file, 'w')

        for i in range(L*n):
            outf.write('ipad_D%d,' % i)
        
        for i in range(L*m):
            outf.write('opad_Z%d,' % i)
        



In [8]:
#生成io.tdf用於ICC
import random

print("Top design File generating")
iovcc_list=[]   #10
iognd_list=[]   #10
corevcc_list=[] #5
coregnd_list=[] #5
number_of_core_power=2
number_of_IO_power=5
for i in range(1,number_of_core_power+1):
    corevcc_list.append("core_vcc"+str(i))
    coregnd_list.append("core_gnd"+str(i))
for i in range(1,number_of_IO_power+1):
    iovcc_list.append("io_vcc"+str(i))
    iognd_list.append("io_gnd"+str(i))

path = '../win_verilog/wc_2_3/'
file = 'io_tdf.txt'
with open(path + file,'r') as f:
    txt =f.read()
    list_txt = list(txt.split(","))  #split port infomation #73
list_txt.remove("")

# print(iovcc_list,iognd_list)
# print(len(list_txt))
totoal_len = len(list_txt)+len(iovcc_list)+len(iognd_list)+len(coregnd_list)+len(corevcc_list)

total_pad=[]

for l in [list_txt,iovcc_list,iognd_list,coregnd_list,corevcc_list]:
    total_pad+=l
print("port len",len(list_txt))
print("IO VCC len",len(iovcc_list))
print("IO GND len",len(iognd_list))
print("CORE VCC len",len(corevcc_list))
print("CORE GND len",len(coregnd_list))
print(len(total_pad))
side_list=[19,19,18,18]
total_pad = random.sample(total_pad,len(total_pad))
# print(total_pad)
file = 'io.tdf'
with open(path + file,"w") as f:
    f.write("\
set_pad_physical_constraints -side 1 -pad_name cornerUL\n\
set_pad_physical_constraints -side 2 -pad_name cornerUR\n\
set_pad_physical_constraints -side 3 -pad_name cornerLR\n\
set_pad_physical_constraints -side 4 -pad_name cornerLL\n\n")

    totoal_pad_idx=0
    for idx,side_size in enumerate(side_list) :
        for order in range(1,side_size+1):
            f.write("set_pad_physical_constraints ")
            f.write("-side "+str(idx+1))
            f.write(" -pad_name ")
            f.write(total_pad[totoal_pad_idx])
            totoal_pad_idx+=1
            f.write("   -order "+str(order))
            f.write(" -min_left_iospace 16 -min_right_iospace 16\n")
        f.write("\n")




Top design File generating
port len 60
IO VCC len 5
IO GND len 5
CORE VCC len 2
CORE GND len 2
74


In [22]:
#將DC要的setup檔案跟script複製到各個資料夾

import shutil
import os
def copy_files_to_folders(source_path, folders):
    # 確保source_path存在
    if not os.path.exists(source_path):
        print(f"錯誤: 路徑 '{source_path}' 不存在")
        return

    # 確保folders是一個存在的資料夾清單
    for folder in folders:
        if not os.path.exists(folder):
            print(f"錯誤: 資料夾 '{folder}' 不存在")
            return

    # 取得source_path下的所有檔案
    files = os.listdir(source_path)

    # 複製檔案到每個目標資料夾
    for folder in folders:
        for file in files:
            source_file_path = os.path.join(source_path, file)
            destination_file_path = os.path.join(folder, file)
            shutil.copy(source_file_path, destination_file_path)
            print(f"已複製檔案 '{file}' 到資料夾 '{folder}'")
        
source_path = '../DC_script'
for n in range(4, 10, 1):
    for r in range(3, 6, 1):
        m = n - r + 1
        if (m<2):
            break
        target_folders = ['../win_verilog/wc_%d_%d/' % (m, r)]
        copy_files_to_folders(source_path, target_folders)

已複製檔案 '.synopsys_dc.setup' 到資料夾 '../win_verilog/wc_2_3/'
已複製檔案 '00_run.tcl' 到資料夾 '../win_verilog/wc_2_3/'
已複製檔案 '01_import.tcl' 到資料夾 '../win_verilog/wc_2_3/'
已複製檔案 '02_compile.tcl' 到資料夾 '../win_verilog/wc_2_3/'
已複製檔案 '03_output.tcl' 到資料夾 '../win_verilog/wc_2_3/'
已複製檔案 '.synopsys_dc.setup' 到資料夾 '../win_verilog/wc_3_3/'
已複製檔案 '00_run.tcl' 到資料夾 '../win_verilog/wc_3_3/'
已複製檔案 '01_import.tcl' 到資料夾 '../win_verilog/wc_3_3/'
已複製檔案 '02_compile.tcl' 到資料夾 '../win_verilog/wc_3_3/'
已複製檔案 '03_output.tcl' 到資料夾 '../win_verilog/wc_3_3/'
已複製檔案 '.synopsys_dc.setup' 到資料夾 '../win_verilog/wc_2_4/'
已複製檔案 '00_run.tcl' 到資料夾 '../win_verilog/wc_2_4/'
已複製檔案 '01_import.tcl' 到資料夾 '../win_verilog/wc_2_4/'
已複製檔案 '02_compile.tcl' 到資料夾 '../win_verilog/wc_2_4/'
已複製檔案 '03_output.tcl' 到資料夾 '../win_verilog/wc_2_4/'
已複製檔案 '.synopsys_dc.setup' 到資料夾 '../win_verilog/wc_4_3/'
已複製檔案 '00_run.tcl' 到資料夾 '../win_verilog/wc_4_3/'
已複製檔案 '01_import.tcl' 到資料夾 '../win_verilog/wc_4_3/'
已複製檔案 '02_compile.tcl' 到資料夾 '../win_verilog/wc_4_3/'
